In [2]:
import tensorflow as tf

out_dir = "logs/embeddings"
ckpt_path = tf.train.latest_checkpoint(out_dir)
if not ckpt_path:
    raise FileNotFoundError(f"No checkpoint found under {out_dir}. "
                            "Check the folder and filenames (e.g. embedding.ckpt-1.*).")
print("Using checkpoint:", ckpt_path)

vars_in_ckpt = tf.train.list_variables(ckpt_path)
print("Variables in checkpoint:")
for name, shape in vars_in_ckpt:
    print(" ", name, shape)

candidate = None
candidate_shape = None
largest_size = 0
for name, shape in vars_in_ckpt:
    if len(shape) == 2:
        size = shape[0] * shape[1]
        if size > largest_size:
            largest_size = size
            candidate = name
            candidate_shape = shape

if candidate is None:
    raise RuntimeError("No 2-D variable found in checkpoint. Check saved checkpoint contents above.")

print(f"Loading variable '{candidate}' with shape {candidate_shape} from checkpoint...")
embeddings = tf.train.load_variable(ckpt_path, candidate)  # returns numpy array
print("Loaded embeddings shape:", embeddings.shape)

Using checkpoint: logs/embeddings\embedding.ckpt-1
Variables in checkpoint:
  _CHECKPOINTABLE_OBJECT_GRAPH []
  embedding/.ATTRIBUTES/VARIABLE_VALUE [30, 384]
  save_counter/.ATTRIBUTES/VARIABLE_VALUE []
Loading variable 'embedding/.ATTRIBUTES/VARIABLE_VALUE' with shape [30, 384] from checkpoint...
Loaded embeddings shape: (30, 384)


In [6]:
import faiss
import numpy as np
import pandas as pd

# Cargar metadata
metadata = pd.read_csv("logs/embeddings/metadata.tsv", sep="\t")

# Normalizar embeddings
embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# Crear índice FAISS
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings_norm.astype("float32"))

def top_k_products(query_idx, k=5):
    query_vec = embeddings_norm[query_idx].astype("float32").reshape(1, -1)
    D, I = index.search(query_vec, k=50)  # pedimos más resultados y luego filtramos

    productos = []
    for dist, idx in zip(D[0], I[0]):
        if metadata.iloc[idx]["tipo"] == "producto":
            productos.append((dist, idx))
        if len(productos) >= k:
            break

    return productos

# Ejemplo: recomendación para el cliente 0
cliente_idx = 1
print("Cliente:", metadata.iloc[cliente_idx]["texto_completo"])
print("\nTop-5 productos similares:")
for dist, idx in top_k_products(cliente_idx, k=5):
    row = metadata.iloc[idx]
    print(f" - {row['label']} | score={dist:.4f}")


Cliente: Carlos Sánchez Ciudad de México México Camiseta Deportiva Ultralight Auriculares Inalámbricos X Zapatillas Urbanas Fit Libro: El Círculo Mágico Set de Ollas Premium Camiseta Deportiva Ultralight Auriculares Inalámbricos X Auriculares Inalámbricos X Auriculares Inalámbricos X Auriculares Inalámbricos X Camiseta Deportiva Ultralight Camiseta técnica de alto rendimiento para deportes. Fabricada con un tejido ligero y transpirable que absorbe la humedad y se seca rápidamente. Su corte atlético y las costuras planas evitan las rozaduras y maximizan la libertad de movimiento. El material es 100% poliéster reciclado, lo que la convierte en una opción sostenible y ecológica. Disponible en varios colores vibrantes. Perfecta para correr, ir al gimnasio o cualquier actividad física intensa. Auriculares Bluetooth de diadema diseñados para audiófilos. Cuentan con controladores de neodimio de 50mm que ofrecen un sonido de alta fidelidad con bajos ricos y agudos cristalinos. La tecnología de

In [9]:
from sentence_transformers import SentenceTransformer

print("SentenceTransformers disponible. Usando modelo multilingüe.")
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
st_model = SentenceTransformer(model_name)


SentenceTransformers disponible. Usando modelo multilingüe.


C:\Users\aml\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aml\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' pac

In [10]:
def search_query(query_text, k=5, tipo="producto"):
    # generar embedding del query
    q_emb = st_model.encode([query_text], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)

    # buscar en FAISS
    D, I = index.search(q_emb.astype("float32"), k=50)

    resultados = []
    for dist, idx in zip(D[0], I[0]):
        if metadata.iloc[idx]["tipo"] == tipo:
            row = metadata.iloc[idx]
            resultados.append((dist, row["label"], row["texto_completo"]))
        if len(resultados) >= k:
            break

    return resultados

In [15]:
# Ejemplo de uso:
query = "libro"
resultados = search_query(query, k=5, tipo="producto")

print(f"🔎 Query: {query}\n")
for score, label, texto in resultados:
    print(f"- {label} | score={score:.4f}\n  {texto[:100]}...")

🔎 Query: libro

- Libro: El Círculo Mágico | score=0.6085
  Libro: El Círculo Mágico...
- Cámara de Acción 4K | score=0.3329
  Cámara de Acción 4K...
- Mochila Urbana Tech | score=0.3230
  Mochila Urbana Tech...
- Zapatillas Urbanas Fit | score=0.3190
  Zapatillas Urbanas Fit...
- Set de Ollas Premium | score=0.2567
  Set de Ollas Premium...
